# devilWASTE 2023

#### Sequncing file issue
This data set has three total sequencing runs. Many samples are present across the multiple sequence runs due to low-quality reads.

Based on several qiime2forum threads, the most prudent solution is below:
- https://forum.qiime2.org/t/samples-run-on-more-than-one-sequencing-run/20391 <- most recent thread

However, previous suggestions have been made for simply retaining the high-read replicates:
- https://forum.qiime2.org/t/merge-or-discard-sequences-from-duplicated-samples-across-multiple-runs/13643

Solution is to run DADA2 separately on each sequencing run (done, due to potential for error modeling) and then merge the output files (both the feature tables and the feature data {i.e., representatative sequence file]). Important to denoise all three datasets with the SAME parameters separately and then merge outputs.

#### Merging the Feature Tables
- Use the command: qiime feature-table merge
- https://docs.qiime2.org/2022.11/plugins/available/feature-table/merge/
- ***Important:*** On merging feature tables step, default option is to rise an error on duplicated samples. Providing 
    'sum' for --p-overlap-method will override this behavior and pool all features from duplicated samples in one, and summarize all identical features in that sample. So at the end you will have more reads than in each of the runs separately.

#### Merging the Feature Sequence Tables
- Use the command: qiime feature-table merge-seqs
- https://docs.qiime2.org/2022.11/plugins/available/feature-table/merge-seqs/

### Import data

Variant of the fastq manifest format: PairedEndFastqManifestPhred33V2

### Run 1

In [ ]:
qiime tools import \
  --type "SampleData[PairedEndSequencesWithQuality]" \
  --input-format PairedEndFastqManifestPhred33V2 \
  --input-path ./manifest_run1.tsv \
  --output-path ./demux_seqs_run1.qza

In [ ]:
qiime demux summarize \
  --i-data ./demux_seqs_run1.qza \
  --o-visualization ./demux_seqs_run1.qzv

#### QC using DADA2

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux_seqs_run1.qza \
  --p-trim-left-f 20 \
  --p-trim-left-r 20 \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 240 \
  --o-table table_run1.qza \
  --o-representative-sequences rep_seqs_run1.qza \
  --o-denoising-stats denoising_stats_run1.qza

In [ ]:
qiime metadata tabulate \
  --m-input-file ./denoising_stats_run1.qza  \
  --o-visualization ./denoising_stats_run1.qzv

### Run 2

In [ ]:
qiime tools import \
  --type "SampleData[PairedEndSequencesWithQuality]" \
  --input-format PairedEndFastqManifestPhred33V2 \
  --input-path ./manifest_run2.tsv \
  --output-path ./demux_seqs_run2.qza

In [ ]:
qiime demux summarize \
  --i-data ./demux_seqs_run2.qza \
  --o-visualization ./demux_seqs_run2.qzv

#### QC using DADA2

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux_seqs_run2.qza \
  --p-trim-left-f 20 \
  --p-trim-left-r 20 \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 240 \
  --o-table table_run2.qza \
  --o-representative-sequences rep_seqs_run2.qza \
  --o-denoising-stats denoising_stats_run2.qza

In [ ]:
qiime metadata tabulate \
  --m-input-file ./denoising_stats_run2.qza  \
  --o-visualization ./denoising_stats_run2.qzv

### Run 3

In [ ]:
qiime tools import \
  --type "SampleData[PairedEndSequencesWithQuality]" \
  --input-format PairedEndFastqManifestPhred33V2 \
  --input-path ./manifest_run3.tsv \
  --output-path ./demux_seqs_run3.qza

In [ ]:
qiime demux summarize \
  --i-data ./demux_seqs_run3.qza \
  --o-visualization ./demux_seqs_run3.qzv

#### QC using DADA2

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux_seqs_run3.qza \
  --p-trim-left-f 20 \
  --p-trim-left-r 20 \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 240 \
  --o-table table_run3.qza \
  --o-representative-sequences rep_seqs_run3.qza \
  --o-denoising-stats denoising_stats_run3.qza

In [ ]:
qiime metadata tabulate \
  --m-input-file ./denoising_stats_run3.qza  \
  --o-visualization ./denoising_stats_run3.qzv

## Merge Feature Tables

In [ ]:
qiime feature-table merge \
    --i-tables table_run1.qza table_run2.qza table_run3.qza \
    --p-overlap-method sum \
    --o-merged-table merged_table.qza

## Merge Representaive Sequences Tables

In [ ]:
qiime feature-table merge-seqs \
    --i-data rep_seqs_run1.qza rep_seqs_run2.qza rep_seqs_run3.qza \
    --o-merged-data merged_rep_seqs.qza

In [ ]:
qiime feature-table summarize \
  --i-table merged_table.qza \
  --o-visualization merged_table.qzv \

qiime feature-table tabulate-seqs \
  --i-data merged_rep_seqs.qza \
  --o-visualization merged_rep_seqs.qzv